# 36120 AT1 Kaggle NBA prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [6]:
train_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\train.csv')
train_data.fillna(0, inplace=True)

numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()

if 'drafted' in numerical_features:
    numerical_features.remove('drafted')

print("Numerical features included:", numerical_features)

Numerical features included: ['GP', 'Min_per', 'Ortg', 'usg', 'eFG', 'TS_per', 'ORB_per', 'DRB_per', 'AST_per', 'TO_per', 'FTM', 'FTA', 'FT_per', 'twoPM', 'twoPA', 'twoP_per', 'TPM', 'TPA', 'TP_per', 'blk_per', 'stl_per', 'ftr', 'porpag', 'adjoe', 'pfr', 'year', 'Rec_Rank', 'ast_tov', 'rimmade', 'rimmade_rimmiss', 'midmade', 'midmade_midmiss', 'rim_ratio', 'mid_ratio', 'dunksmade', 'dunksmiss_dunksmade', 'dunks_ratio', 'pick', 'drtg', 'adrtg', 'dporpag', 'stops', 'bpm', 'obpm', 'dbpm', 'gbpm', 'mp', 'ogbpm', 'dgbpm', 'oreb', 'dreb', 'treb', 'ast', 'stl', 'blk', 'pts']


C:\Users\13803\AppData\Local\Temp\ipykernel_16504\3558450572.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\train.csv')


In [7]:
X = train_data[numerical_features]
y = train_data['drafted']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

from xgboost import XGBClassifier
xgb_model = XGBClassifier(
    max_depth=6,
    n_estimators=100,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,
    scale_pos_weight=1
)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [8]:
y_val_pred = xgb_model.predict(X_val)
y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
auc = roc_auc_score(y_val, y_val_proba)

print("Validation Accuracy: {:.5f}".format(accuracy))
print("Validation Precision: {:.5f}".format(precision))
print("Validation Recall: {:.5f}".format(recall))
print("Validation F1 Score: {:.5f}".format(f1))
print("Validation AUC Score: {:.5f}".format(auc))

Validation Accuracy: 0.99324
Validation Precision: 0.72941
Validation Recall: 0.60784
Validation F1 Score: 0.66310
Validation AUC Score: 0.99747


In [14]:
import pandas as pd
from xgboost import XGBClassifier

test_data = pd.read_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\test.csv')
test_data.fillna(0, inplace=True)  

numerical_features = test_data.select_dtypes(include=[np.number]).columns.tolist()
if 'drafted' in numerical_features: 
    numerical_features.remove('drafted')

X_test = test_data[numerical_features]




In [15]:
missing_cols = set(X_train.columns) - set(X_test.columns)
for c in missing_cols:
    X_test[c] = 0 

X_test = X_test[X_train.columns]

y_test_proba = xgb_model.predict_proba(X_test)[:, 1]

In [16]:
predictions = pd.DataFrame({
    'player_id': test_data['player_id'],  
    'draft_probability': y_test_proba
})
predictions.to_csv('C:\\Users\\13803\\Downloads\\uts-36120-2024-spr\\test_predictions.csv', index=False)

print("Predictions have been saved to 'test_predictions.csv'")


Predictions have been saved to 'test_predictions.csv'
